In [2]:
from robobrowser import RoboBrowser
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

def get_number(s):
    try:
        return float(s.replace(",",""))
    except ValueError:
        return None

#Open URL and create RoboBrowser Instance
browser = RoboBrowser(parser='html.parser', history=False) 
#2016-01-02, 366 periods
dates = pd.date_range('2016-01-02', periods=366, freq='2D').format('YYYY-MM-dd')[1:] # url parameter is last date shown on website, website shows two dates for one request
print(dates)
results = pd.DataFrame()

for d in dates:
    url='https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/' + d + '/DE'
    print(url)
    browser.open(url)  
    soup = bs(str(browser.parsed), 'html.parser') # get html
    dayData = soup.find("table") # select table
    #print(soup.prettify())
    myDates = pd.date_range(pd.to_datetime(d) - pd.to_timedelta(1, 'D'), periods=2, freq="D") # create two shown dates
    tableHeader = dayData.find("tr", class_=None) # get table header row
    columnNames = [p.get_text() for p in tableHeader.findAll("th")][2:11]
    priceRows = dayData.findAll("tr", class_=None)[1:] # get rows with €/Mwh with hourly interval
    hours = ["'00 - 01", "'01 - 02", "'02 - 03", "'03 - 04", "'04 - 05", "'05 - 06", 
             "'06 - 07", "'07 - 08", "'08 - 09", "'09 - 10", "'10 - 11", "'11 - 12", 
             "'12 - 13", "'13 - 14", "'14 - 15", "'15 - 16", "'16 - 17", "'17 - 18", 
             "'18 - 19", "'19 - 20", "'20 - 21", "'21 - 22", "'22 - 23", "'23 - 24"] # table column names; "'" at beginning for opening in excel

    print(d)
    if d in ["2016-10-30", "2017-10-29"]: # two rows for 2-3am period for day of "Zeitumstellung auf Winterzeit", only keep first
        del priceRows[3]
    # get prices for both days on website
    prices = np.zeros((2, 24, 9))
    for i in range(24):
        prices[0][i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")][3:12]
        prices[1][i] = [get_number(p.get_text()) for p in priceRows[i].findAll("td")][13:]
    #print(prices)
    for i in range(2):
        df = pd.DataFrame(prices[i])
        df.columns = columnNames
        df["hours"] = hours
        df = df.set_index("hours", drop=False)
        df["date"] = myDates[i]
        if results.empty:
            results = df
        else:
            results = results.append(df)
        if d == "2018-01-01":
            results = results[:-24] #cut off 1.1.2018
        #print(df)
results = results.set_index(["date", "hours"])

print(results)
results.to_csv("../00_data/intraday.csv", sep=";", decimal=",")


['2016-01-02', '2016-01-04', '2016-01-06', '2016-01-08', '2016-01-10', '2016-01-12', '2016-01-14', '2016-01-16', '2016-01-18', '2016-01-20', '2016-01-22', '2016-01-24', '2016-01-26', '2016-01-28', '2016-01-30', '2016-02-01', '2016-02-03', '2016-02-05', '2016-02-07', '2016-02-09', '2016-02-11', '2016-02-13', '2016-02-15', '2016-02-17', '2016-02-19', '2016-02-21', '2016-02-23', '2016-02-25', '2016-02-27', '2016-02-29', '2016-03-02', '2016-03-04', '2016-03-06', '2016-03-08', '2016-03-10', '2016-03-12', '2016-03-14', '2016-03-16', '2016-03-18', '2016-03-20', '2016-03-22', '2016-03-24', '2016-03-26', '2016-03-28', '2016-03-30', '2016-04-01', '2016-04-03', '2016-04-05', '2016-04-07', '2016-04-09', '2016-04-11', '2016-04-13', '2016-04-15', '2016-04-17', '2016-04-19', '2016-04-21', '2016-04-23', '2016-04-25', '2016-04-27', '2016-04-29', '2016-05-01', '2016-05-03', '2016-05-05', '2016-05-07', '2016-05-09', '2016-05-11', '2016-05-13', '2016-05-15', '2016-05-17', '2016-05-19', '2016-05-21', '2016

2016-03-04
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-06/DE
2016-03-06
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-08/DE
2016-03-08
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-10/DE
2016-03-10
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-12/DE
2016-03-12
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-14/DE
2016-03-14
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-16/DE
2016-03-16
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-18/DE
2016-03-18
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-20/DE
2016-03-20
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-22/DE
2016-03-22
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-03-24/DE
2016-03-24

2016-08-17
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-19/DE
2016-08-19
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-21/DE
2016-08-21
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-23/DE
2016-08-23
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-25/DE
2016-08-25
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-27/DE
2016-08-27
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-29/DE
2016-08-29
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-08-31/DE
2016-08-31
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-09-02/DE
2016-09-02
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-09-04/DE
2016-09-04
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2016-09-06/DE
2016-09-06

2017-01-30
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-01/DE
2017-02-01
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-03/DE
2017-02-03
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-05/DE
2017-02-05
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-07/DE
2017-02-07
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-09/DE
2017-02-09
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-11/DE
2017-02-11
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-13/DE
2017-02-13
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-15/DE
2017-02-15
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-17/DE
2017-02-17
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-02-19/DE
2017-02-19

2017-07-15
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-17/DE
2017-07-17
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-19/DE
2017-07-19
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-21/DE
2017-07-21
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-23/DE
2017-07-23
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-25/DE
2017-07-25
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-27/DE
2017-07-27
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-29/DE
2017-07-29
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-07-31/DE
2017-07-31
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-08-02/DE
2017-08-02
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-08-04/DE
2017-08-04

2017-12-28
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2017-12-30/DE
2017-12-30
https://www.epexspot.com/en/market-data/intradaycontinuous/intraday-table/2018-01-01/DE
2018-01-01
                     Low(€/MWh)  High(€/MWh)  Last(€/MWh)  \
date       hours                                            
2016-01-01 '00 - 01       23.00        38.00        36.00   
           '01 - 02       17.15        30.00        24.99   
           '02 - 03       12.00        35.00        32.85   
           '03 - 04       13.00        32.00        20.00   
           '04 - 05       14.00        34.00        30.00   
           '05 - 06       14.00        35.00        34.00   
           '06 - 07       12.00        34.00        25.11   
           '07 - 08       12.90        37.50        36.00   
           '08 - 09       17.73        41.00        31.90   
           '09 - 10       12.00        44.00        14.00   
           '10 - 11       19.27        46.00        30.00  